In [1]:
import pandas as pd

In [35]:
data = pd.read_csv('users_history.csv')

In [12]:
data.shape

(103723, 13)

In [16]:
data.shape

(107367, 13)

In [29]:
data.shape

(160549, 13)

In [36]:
data.user_id.nunique()

1150